In [1]:
import requests
import pandas as pd
import json
import time
from tqdm import tqdm
from vt_api_key import api_key

In [86]:
train_data = pd.read_csv('data/processed/sample_train.csv')

C:\Users\ilanv\AppData\Local\Temp/ipykernel_31604/1024237520.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('data/processed/sample_train.csv')


In [92]:
train_malwares = pd.DataFrame(train_data[train_data['label'] == 1]['sha256'].copy())
train_malwares['status_code'] = 0
train_malwares['timestamp'] = None
train_malwares['file_number'] = -1

In [117]:
# DO NOT RUN THIS CODE!
# train_malwares.to_csv('data/virustotal/train_file_status.csv', index=False)

In [118]:
# DO NOT RUN THIS CODE!
# with open('data/virustotal/train_file_data.json', 'w') as f:
    # json.dump({}, f)

In [13]:
# index of last file that wasn't checked
last_index = train_malwares[train_malwares['status_code'] == 0].index[0]
last_index

1001

In [8]:
test_data = pd.read_csv('data/processed/sample_test.csv')

C:\Users\ilanv\AppData\Local\Temp/ipykernel_14956/3229735755.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('data/processed/sample_test.csv')


In [9]:
test_malwares = pd.DataFrame(test_data[test_data['label'] == 1]['sha256'].copy())
test_malwares['status_code'] = 0
test_malwares['timestamp'] = None
test_malwares['file_number'] = -1

In [11]:
# DO NOT RUN THIS CODE!
# test_malwares.to_csv('data/virustotal/test_file_status.csv', index=False)

In [4]:
test_data

,sha256,md5,appeared,label,avclass,size,vsize,has_debug,exports,imports,...,avlength,printables,entropy,paths,urls,registry,MZ,status_code,timestamp,file_number
0,8d32f3535d31795886bf62523ec9035d843ef0e2c7032b...,5714e1c1af7c89ff73e397398868632c,2018-11,0,NaN,45568,65536,0,4,108,...,13.124611,4213,5.744219,0,0,0,1,0,None,-1
1,f9fbcd42e4dc36bec2fb093b8e7479211def628553e2ac...,9ed302dc291bd0062ac8b4aa588e9012,2018-11,0,NaN,167936,262144,0,0,0,...,6.963756,4227,6.415260,0,0,0,2,0,None,-1
2,f5bbd3a215e4e62e12ad43b6e05e3bbae2ae5632910cbd...,035c0ebe04d3803aae1f36f02c810441,2018-11,0,NaN,817518,3952640,0,0,0,...,18.224425,97446,5.710346,0,2,14,4,0,None,-1
3,fb937d1d32cb75e41e162f694fe3ba77ada98ae4e7f44b...,9694d64e55df2a75e080a57aadbd8977,2018-11,0,NaN,16384,32768,0,0,1,...,11.139785,2072,5.036963,0,0,0,1,0,None,-1
4,c8dd919a9fe6d27e6d30333a0ce55c4bed602f7d51f79d...,38cd23930e6c759ed42ae7b7935c4154,2018-11,0,NaN,773044,25337856,1,0,0,...,7.841193,25774,6.406318,0,0,0,4,0,None,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,5af09459d1bddc385677b3d1019fdfd3fcfc3530c1c561...,fe3b194f5e0f46cbfae46cd192613c83,2018-11,1,sality,123392,131072,0,0,11,...,5.910448,2772,6.545131,0,0,0,1,0,None,-1
119996,9d1a96817df0c23a64347bc9548146fc3a801b8f7b1170...,2e9c3ea6f93cb46e967b911c65ec774c,2018-11,1,high,309760,327680,0,0,3,...,5.980769,6842,6.530765,0,0,0,1,0,None,-1
119997,4f93bd34cdfb450b0705043170daa916739c6b3044b6ff...,aa41f376a8af983f1aab1e16e4f6259c,2018-11,1,emotet,222741,856064,1,0,20,...,7.735366,6343,3.669333,0,0,0,6,0,None,-1
119998,c16133fcc74483a984ed72f53dffa6721ad013a6e59a65...,533728b2d6e4b87596205e36479917a4,2018-12,1,sality,294726,266240,0,0,119,...,8.412698,7950,5.879615,2,0,0,1,0,None,-1


In [27]:
file_hash = 'moti'
url = f"https://www.virustotal.com/api/v3/files/{file_hash}"

headers = {
    "accept": "application/json",
    "x-apikey": "5adf3682d43ee1cb47ffc91c8e04e520be5637f4df6a1fee1190d964b806f4fa"
}

response = requests.get(url, headers=headers)

print(response.text)

{"error":{"code":"NotFoundError","message":"Resource not found."}}



In [23]:
def get_remaning_daily_quota(api_key):
    url = f"https://www.virustotal.com/api/v3/users/{api_key}/overall_quotas"

    headers = {
        "accept": "application/json",
        "x-apikey": api_key
    }

    response = requests.get(url, headers=headers)
    user_daily_quota = response.json()['data']['api_requests_daily']['user']
    used = user_daily_quota['used']
    allowed = user_daily_quota['allowed']
    return allowed - used

In [14]:
url = f"https://www.virustotal.com/api/v3/users/{api_key}/overall_quotas"

headers = {
    "accept": "application/json",
    "x-apikey": api_key
}

response = requests.get(url, headers=headers)

In [56]:
get_remaning_daily_quota(api_key)

19936

In [113]:
def create_batches(start_index, df_size):
    # num_avilable_requests = get_remaning_daily_quota(api_key)
    num_avilable_requests = 20000
    num_requests = 0 
    batches = []
    while num_requests < num_avilable_requests and start_index < df_size:
        batch_samples = min([3, num_avilable_requests - num_requests, df_size - start_index])
        end_index = start_index + batch_samples 
        batches.append(range(start_index, end_index))
        num_requests += 3
        start_index = end_index
    return batches

In [81]:
batches = create_batches(train_malwares, 0)
batches[0]

range(0, 3)

In [82]:
data_json = {}
batches = create_batches(train_malwares, 0)
for b in range(len(batches)):
    for idx in tqdm(batches[b], desc = f"batch {b}"):
        
        sha256 = train_malwares.loc[idx, 'sha256']
        # get response
        url = f"https://www.virustotal.com/api/v3/files/{sha256}"
        headers = {
            "accept": "application/json",
            "x-apikey": api_key
        }
        response = requests.get(url, headers=headers)
        # update status and timestamp
        train_malwares.loc[idx, 'timestamp'] = pd.Timestamp.now()
        train_malwares.loc[idx, 'status_code'] = response.status_code
        # save response json
        data_json[sha256] = response.json()

    # save progress
    train_malwares.to_csv('data/virustotal/train_file_status.csv', index=False)
    with open('data/virustotal/train_file_data.json', 'w') as f:
            json.dump(data_json, f)

    if b != len(batches) - 1:
        # sleep for 1 minute
        time.sleep(3)



        

batch 3: 100%|██████████| 2/2 [00:01<00:00,  1.67it/s]


In [85]:
# open json file
with open('data/virustotal/train_file_data.json', 'r') as f:
    data_json_shmulik = json.load(f)
len(data_json_shmulik)

11

In [19]:
# get number of malicious and undected votes 
malicious_votes = response.json()['data']['attributes']['last_analysis_stats']['malicious']
undetected_votes = response.json()['data']['attributes']['last_analysis_stats']['undetected']
response.json()['data']['attributes']['last_analysis_stats']

{'harmless': 0,
 'type-unsupported': 4,
 'suspicious': 0,
 'confirmed-timeout': 0,
 'timeout': 0,
 'failure': 5,
 'malicious': 53,
 'undetected': 9}

In [21]:
json_dic = {}
json_dic[file_hash] = response.json()
# convert to json 
json_dic = json.dumps(json_dic) 

In [28]:
# get response code
response.status_code

404

In [114]:
len(train_malwares)
batches = create_batches(79950, len(train_malwares))

In [119]:
get_remaning_daily_quota(api_key)

18863